This is the simulation and test codes for Timeseries DP synthesizing method

In [ ]:
import pandas as pd
import numpy as np
from scipy import interpolate

: 

build function for adding windows in a timeseries

In [5]:
# add windows function

def add_windows(timeseries:np.array, window_size:int):
    '''
    This function adds window in the front and back of a given timeseries.
    The amount of window size is added.
    '''
    
    # first fill the added points with Na values 
    timeseries_list = timeseries.tolist()
    added_windows = np.repeat(np.nan, window_size).tolist()
    inserted_timeseries = np.array(added_windows + timeseries_list + added_windows)

    # now we interpolate the Na values
    xs = [x for x in range(len(inserted_timeseries))]
    f = interpolate.interp1d(xs[window_size:-window_size], timeseries_list, fill_value='extrapolate')
    interpolated_timeseries = f(xs)
    
    return interpolated_timeseries

import sample

In [9]:
df = pd.read_csv('../data/processed/live_df_interpolated.csv')

df.head()

sample_sequence = df[df.subject_id == 10001884]

In [13]:
add_windows(sample_sequence['heart_rate'],1)
# It works!

array([71., 68., 65., 63., 63., 66., 66., 65., 66., 66., 67., 66., 63.,
       64., 70., 71., 74., 74., 72., 74., 77., 87., 75., 74., 73.])

Now we need to make a module for the BLM which is the boundary feeder

In [42]:
!tree timedp

/bin/bash: tree: command not found


In [ ]:
# %%writefile ../timedp/mechanism/Numbers.py

class Numbers:
    def __init__(self, value:float):
        self.value = value

    def __repr__(self):
        return f"Numbers(type=={self.number_type})"

    def __str__(self):
        return self.value

class Positive(Numbers):
    def __init__(self, value):
        super().__init__(value)
        self.number_type = "positive"
    
class Negative(Numbers):
    def __init__(self, value):
        super().__init__(value)
        self.number_type = "negative"

class Zero(Numbers):
    def __init__(self, value):
        super().__init__(value)
        self.number_type = "zero"


Zero(0)

: 

In [91]:
# %%writefile ../timedp/mechanism/Trend.py

class Trend:
    '''
    This class is a class that defines trend type
    '''
    def __init__(self,value_before, value, value_after, time_window):
        self.value_before = value_before
        self.value = value
        self.value_after = value_after
        self.time_window = time_window

        if time_window <= 0 :
            raise ValueError('time window should be a positive integer!')

        self.trend = self._define_trend()
    
    def _define_trend_type(self, value):

        if value == 0 :
            return Zero(value)
        elif value > 0 :
            return Positive(value)
        else :
            return Negative(value)

    
    def _define_trend(self):
        between_value_before_and_value =  (self.value-self.value_before)/self.time_window
        between_value_after_and_value = (self.value_after-self.value)/self.time_window
    
        trend = tuple(list(map(self._define_trend_type,[between_value_before_and_value, between_value_after_and_value])))
        return trend
    

Trend(2,3,1,1).trend
# the trend between t-1, t and t, t+1

Writing ../timedp/mechanism/Trend.py


In [37]:
# %%writefile ../timedp/boundary_feeder.py

class BoundaryFeeder:
    '''
    This object is a boundary feeder used when the Timeseries DP algorithm is onging.
    This class give the boundary information to the BLM
    '''

    def __init__(self, timeseries : np.array, window_size: int):
        self.timeseries = timeseries
        self.window_size = window_size
        self.boundary_information = self._make_boundary_information() 
    
    def _get_boundary(self, idx):
        value = self.timeseries[idx]
        v_before, v_after = self.timeseries[idx - self.window_size], self.timeseries[idx + self.window_size]
        return v_before, v_after

    def _make_boundary_information(self):
        theLength = len(self.timeseries)
        idx_list = [idx for idx in range(self.window_size, theLength-self.window_size)]
        boundary_list = map(self._get_boundary, idx_list)

        return {idx : (v_before, v_after) for idx, (v_before, v_after) in zip(idx_list, boundary_list)}

    def _check_trend(self, value, boundary:tuple):
        '''
        checks the trend of t-window_size, t, t+window_size
        classifies the the trend
        '''
        # case increasing or decreasing
        

    # def _feed(self):
        


Overwriting ../timedp/boundary_feeder.py


In [35]:
myFeeder = BoundaryFeeder(sample_sequence['heart_rate'], 1)
myFeeder.boundary_information

{1: (68.0, 63.0),
 2: (65.0, 63.0),
 3: (63.0, 66.0),
 4: (63.0, 66.0),
 5: (66.0, 65.0),
 6: (66.0, 66.0),
 7: (65.0, 66.0),
 8: (66.0, 67.0),
 9: (66.0, 66.0),
 10: (67.0, 63.0),
 11: (66.0, 64.0),
 12: (63.0, 70.0),
 13: (64.0, 71.0),
 14: (70.0, 74.0),
 15: (71.0, 74.0),
 16: (74.0, 72.0),
 17: (74.0, 74.0),
 18: (72.0, 77.0),
 19: (74.0, 87.0),
 20: (77.0, 75.0),
 21: (87.0, 74.0)}

Finally, we make our time DP synthesizer

In [ ]:
class TimeDPSynthesizer():

    def __init__(self, timeseries : np.array, feeder: object,epsilon: float, delta: float):
        self.timeseries = timeseries
        self.feeder = feeder
        self.epsilon = epsilon
        self.delt = delta

    def _        


# Imaginary Code

The code that I want to make is something like this..

```{python}
list_of_arrays = [array1, array2, array3...]
synthesized_array_list = []
for array in list_of_arrays:
    myFeeder = BoundaryFeeder(array, window_size = 1)
    synthesizer = TimeDPSynthesizer(myFeeder)

    synthesized_array = synthesizer.synthesize
    synthesized_array_list.append(synthesized_array)
```

We implement this code in below